In [1]:
# import packages
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
from scipy import stats
import statistics
import plotly.graph_objects as go
from scipy.stats import norm
from plotly.subplots import make_subplots
from sklearn.neighbors import KernelDensity

In [2]:
# Load the market prices
S_P100index = pd.read_excel("/Users/dantoni/HW4/TP4.xls", sheet_name = 'S&P100Index', skiprows = 4)
S_P100index.columns = ['date','S&P100','S&P500']
S_P100index = S_P100index.set_index('date')

# Load the stocks prices
S_P100const = pd.read_excel("/Users/dantoni/HW4/TP4.xls", sheet_name = 'S&P100Constituents', skiprows = 3)
S_P100const = S_P100const.drop([0])
S_P100const.rename(columns = {'Name': 'date'}, inplace=True)
S_P100const = S_P100const.set_index('date')
S_P100const = S_P100const.astype(float)

# Load the risk free rates
TBill3Months = pd.read_excel("/Users/dantoni/HW4/TP4.xls", sheet_name = 'TBill3Months', skiprows = 4)
TBill3Months.columns = ['date','US bill 3m']
TBill3Months = TBill3Months.set_index('date')

# Load something
FamaFrenchPortfolios = pd.read_excel("/Users/dantoni/HW4/TP4.xls", sheet_name = 'FamaFrenchPortfolios', skiprows = 21)
FamaFrenchPortfolios.columns = ['date','Small_Low BE/ME', 'Small_Med BE/ME', 'Small_High BE/ME','Big_Low BE/ME', 'Big_Med BE/ME', 'Big_High BE/ME']
FamaFrenchPortfolios = FamaFrenchPortfolios.set_index('date')

In [3]:
S_P100index.iloc[0:-1,0:].values

array([[192.13, 422.88],
       [192.31, 423.61],
       [195.36, 429.19],
       ...,
       [447.88, 882.5 ],
       [451.2 , 885.76],
       [460.55, 902.65]])

In [4]:
# Compute Arithmetic return for each market
AR_100 = pd.DataFrame((S_P100index.iloc[1:,0:].values - S_P100index.iloc[0:-1,0:].values) / S_P100index.iloc[0:-1,0:].values,  columns = ['S&P100','S&P500'])

# Compute Arithmetic return for each stocks
names_company = S_P100const.columns
AR_100_const = pd.DataFrame((S_P100const.iloc[1:,0:].values - S_P100const.iloc[0:-1,0:].values) / S_P100const.iloc[0:-1,0:].values, columns = names_company)

# Compute risk free rates (since it's given annualy we have to divide it by 100 and 52 to get weekly's rates)
AR_US3M = pd.DataFrame((TBill3Months.iloc[1:,0:].values / (100*52)),  columns = ['US 3 month'])

# Test of the CAPM : Time Series approach

## Exercice 1

In [5]:
# Compute the index of the return of interest
index_start = int(np.where(S_P100index.index == '1992-11-12')[0])
index_end = int(np.where(S_P100index.index == '2001-08-16')[0] - 1)

In [6]:
# Compute the beta of each stock thanks to the formula cov(ri,rm)/var(rm)
beta_stock = np.ones(AR_100_const.shape[1])
for i in range(0, AR_100_const.shape[1]):
    beta_stock[i] = np.cov(AR_100_const.iloc[index_start:index_end, i].values, AR_100.iloc[index_start:index_end, 1].values)[0, 1] / np.cov(AR_100_const.iloc[index_start:index_end, i].values, AR_100.iloc[index_start:index_end, 1].values)[1, 1]

pd.DataFrame(beta_stock, index = names_company, columns = ['Beta'])

,Beta
AES,1.012828
ALCOA,0.616923
ALLEGHENY TECHS.,0.702266
AMER.ELEC.PWR.,0.262885
AMER.EXPRESS,1.501120
...,...
WELLS FARGO & CO,1.124522
WEYERHAEUSER,0.713086
WILLIAMS COS.,0.817679
XEROX,1.349616


## Exercice 2

In [7]:
# Compute the alphas and betas of each stock thanks to the linear regression
nb_asset = len(AR_100_const.columns) # Compute the number of colums to deal with
betas = np.zeros(nb_asset) # Initialize an array with nb_asset entries to receive the beta
alphas = np.zeros(nb_asset) # Initialize an array with nb_asset entries to receive the alpha
std_alpha = np.zeros(nb_asset) # Initialize an array with nb_asset entries to receive the std of alpha

for i in range(0, nb_asset):
    zi =  AR_100_const.iloc[index_start:index_end, i].values - AR_US3M.iloc[index_start:index_end, 0].values
    zm = AR_100.iloc[index_start:index_end, 1].values - AR_US3M.iloc[index_start:index_end, 0].values
    X = sm.add_constant(zm)
    y = zi
    
    # Fit regression on the data of interest and save the alpha, beta and std of alpha
    reg = sm.OLS(endog = y, exog = X)
    results = reg.fit()
    betas[i] = results.params[1]
    alphas[i] = results.params[0]
    std_alpha[i] = results.bse[0]

In [8]:
# Compute the t-value of the alpha --> alpha/std_alpha (if absolut value < 1.96 --> can not reject H0 (alpha = 0))
pd.DataFrame(np.transpose(np.array([betas, alphas, std_alpha, np.around(abs(alphas/std_alpha), decimals=2)])), index = names_company, columns = ['beta', 'alpha', 'std_alpha', 't-value'])

,beta,alpha,std_alpha,t-value
AES,1.012350,0.004353,0.002591,1.68
ALCOA,0.616899,0.002267,0.002010,1.13
ALLEGHENY TECHS.,0.703382,-0.001898,0.002084,0.91
AMER.ELEC.PWR.,0.262135,-0.000062,0.001372,0.05
AMER.EXPRESS,1.501551,0.001655,0.001557,1.06
...,...,...,...,...
WELLS FARGO & CO,1.124238,0.001355,0.001535,0.88
WEYERHAEUSER,0.713880,-0.000244,0.001858,0.13
WILLIAMS COS.,0.817565,0.002626,0.002032,1.29
XEROX,1.351780,-0.001567,0.002970,0.53


In [9]:
# Compute the number of stocks (over 95 = nb_asset) with alpha = 0 (statistically speaking)
np.sum(abs(alphas/std_alpha) < 1.96)

90

In [10]:
# Fit regression on the remaining data and save the r-squared of each stocks
index_start2 = int(np.where(S_P100index.index == '2001-08-16')[0] - 1)
r2 = np.zeros(nb_asset)

for i in range(0, nb_asset):
    zi =  AR_100_const.iloc[index_start2:, i].values - AR_US3M.iloc[index_start2:, 0].values
    zm = AR_100.iloc[index_start2:, 1].values - AR_US3M.iloc[index_start2:, 0].values
    X = sm.add_constant(zm)
    y = zi
    
    reg = sm.OLS(endog = y, exog = X, missing='drop')
    results = reg.fit()
    r2[i] = results.rsquared

In [11]:
# Compute the r-squared for each stocks
pd.DataFrame(np.around(r2, decimals=2), index = names_company, columns = ['R-squared']) 

,R-squared
AES,0.19
ALCOA,0.64
ALLEGHENY TECHS.,0.39
AMER.ELEC.PWR.,0.05
AMER.EXPRESS,0.73
...,...
WELLS FARGO & CO,0.39
WEYERHAEUSER,0.53
WILLIAMS COS.,0.15
XEROX,0.29


In [12]:
# Compute the average r-squared of each stocks
statistics.mean(np.around(r2, decimals=2))

0.32789473684210524